## Import Relevant Libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import nltk
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [ ]:
#dataset from kaggle: kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading the Dataset

In [3]:
df=pd.read_csv("IMDB Dataset.csv")
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [5]:
#Checking the sentiment count

print("postive ",np.sum(df["sentiment"]=="positive"))
print("negative ",np.sum(df["sentiment"]=="negative"))

postive  25000
negative  25000


In [6]:
# thus we see the dataset is balanced

In [7]:
def clear_noise(text):
  
  # removing html scripts using Beautiful Soup
  text1=BeautifulSoup(text)
  text1=text1.get_text()

  #removing all other character than alphabets and numbers using regular expression
  text2=re.sub('[^a-zA-Z0-9\s]', '', text1)

  return text2

df["review"]=df["review"].apply(clear_noise)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [8]:
# All html and other special characters is removed now

## Preprocessing the Data 

In [9]:
# Now preprocessing the data according to the bag of word model

In [10]:
# 1. Tokenisation (Breaking down the document into words)

def tokenize(text):
  return text.split()

# 2. Stopword Removal (Removal of words which are not meaningful for the model)

sw=set(nltk.corpus.stopwords.words('english'))

def stopword_removal(text):
  useful_words=[w for w in text if w not in sw]
  return useful_words

# 3. Lemmatization (Changing all forms of a verb to root form like plays ,played etc. to play)

wn=nltk.stem.WordNetLemmatizer()

def lemmatize(text):
  words=[wn.lemmatize(w) for w in text]
  return words

# 4. Building a vocabulary (Each sentence will have a feature vector)

def myTokenizer(text):
  text1=tokenize(text.lower())
  text1=stopword_removal(text1)
  text1=lemmatize(text1)
  return text1

cv=CountVectorizer(tokenizer=myTokenizer)

In [11]:
print(sw)

{'do', 'only', 'those', "haven't", 'whom', 'who', "it's", 'through', 'her', 'where', 'other', "isn't", 'mustn', 'as', 'such', 'with', 'so', 'into', 'd', 'about', 'being', 'of', 'did', "shouldn't", 'hadn', 'an', 'further', 'does', "won't", 'doing', 'down', "mustn't", 'had', 'not', 'again', 'it', 'that', "hadn't", 'hers', 'i', "she's", 'them', 'yourselves', 'from', 'itself', 'off', 'between', 'ain', 'while', 'o', 'to', 'herself', 'we', 'when', 'its', 'were', 'they', 'this', 'haven', 'he', 'my', 'there', 'didn', 'needn', 'which', 'their', 'out', 'all', 'me', 'am', 'theirs', 'what', "weren't", 'up', 'aren', 'isn', 'any', 'is', "didn't", 'should', 'are', 'why', "shan't", 'has', 'because', 'yours', 'here', 'if', 'how', 'after', 'before', 're', "you'd", 'll', 'by', "you'll", 'now', 'hasn', 'until', 'own', 'him', 'themselves', 'nor', 've', 'in', "aren't", "you're", 'more', 'be', 'shan', 'she', 'each', 'too', 'over', "don't", 'or', 'no', 'just', 'have', 'few', 'been', 'ours', 'was', 'then', 'sh

## Training the model 

### Splitting the Dataset into Training Set and Test Set

In [54]:
train_x=df.review[:40000]
train_y=df.sentiment[:40000]

test_x=df.review[40000:]
test_y=df.sentiment[40000:]

print(train_x.shape,type(train_x))
print(test_x.shape,type(test_x))

(40000,) <class 'pandas.core.series.Series'>
(10000,) <class 'pandas.core.series.Series'>


In [55]:
vectorised_data=cv.fit_transform(train_x)
print(vectorised_data.shape)

(40000, 181861)


### Converting the test and train data in vectorised form

In [56]:
cv_train=cv.transform(train_x)
cv_test=cv.transform(test_x)

In [57]:
print(cv_train.shape)

(40000, 181861)


In [110]:
# Labelling the sentiment data

lb=LabelBinarizer()
sentiment_data=lb.fit_transform(df['sentiment'])
print(sentiment_data.shape)

train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]

(50000, 1)


### Building the MLP (Multi layer perceptron) Model

In [111]:
model=Sequential()
model.add(Dense(16,activation="relu",input_shape=(181861,)))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 16)                2909792   
_________________________________________________________________
dense_19 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 17        
Total params: 2,910,081
Trainable params: 2,910,081
Non-trainable params: 0
_________________________________________________________________


In [113]:
hist=model.fit(cv_train,train_sentiments,epochs=60,batch_size=512)

Epoch 1/60
79/79 [==============================] - 1s 19ms/step - loss: 0.5857 - accuracy: 0.8024
Epoch 2/60
79/79 [==============================] - 1s 18ms/step - loss: 0.5283 - accuracy: 0.8247
Epoch 3/60
79/79 [==============================] - 1s 19ms/step - loss: 0.4826 - accuracy: 0.8363
Epoch 4/60
79/79 [==============================] - 1s 18ms/step - loss: 0.4461 - accuracy: 0.8454
Epoch 5/60
79/79 [==============================] - 1s 17ms/step - loss: 0.4165 - accuracy: 0.8528
Epoch 6/60
79/79 [==============================] - 1s 17ms/step - loss: 0.3926 - accuracy: 0.8587
Epoch 7/60
79/79 [==============================] - 1s 19ms/step - loss: 0.3727 - accuracy: 0.8649
Epoch 8/60
79/79 [==============================] - 1s 17ms/step - loss: 0.3560 - accuracy: 0.8696
Epoch 9/60
79/79 [==============================] - 1s 18ms/step - loss: 0.3415 - accuracy: 0.8749
Epoch 10/60
79/79 [==============================] - 1s 19ms/step - loss: 0.3290 - accuracy: 0.8784
Epoch 11/

## Accuracy of the trained model

In [114]:
model.evaluate(cv_test,test_sentiments)

313/313 [==============================] - 0s 1ms/step - loss: 0.2925 - accuracy: 0.8893


[0.2924994230270386, 0.8892999887466431]

In [115]:
# the model works with an accuracy of around 89 percent